In [1]:
from datasets import load_dataset

ds = load_dataset("atmansingh/ljspeech")

/home/khj6051/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'file_id', 'original_transcription'],
        num_rows: 11775
    })
    validation: Dataset({
        features: ['audio', 'text', 'file_id', 'original_transcription'],
        num_rows: 1309
    })
})

In [ ]:
from utils.feature import TorchAudioFbank, TorchAudioFbankConfig
fbank = TorchAudioFbank(config=TorchAudioFbankConfig(sampling_rate=24000, n_mels=100, n_fft=1024, hop_length=256))

ds['train'][0]['audio']['array']

{'audio': {'path': 'LJ042-0181.wav',
  'array': array([0.00189209, 0.00311279, 0.00411987, ..., 0.00213623, 0.00204468,
         0.00131226], shape=(145565,)),
  'sampling_rate': 22050},
 'text': 'Oswald realized that, quote, their thinking and education will be steeped in the traditions of those systems',
 'file_id': 'LJ042-0181',
 'original_transcription': 'Oswald realized that, quote, their thinking and education will be steeped in the traditions of those systems'}

In [ ]:
import librosa
import torch
from utils.feature import TorchAudioFbank, TorchAudioFbankConfig
import time
import matplotlib.pyplot as plt
import numpy as np

fbank = TorchAudioFbank(config=TorchAudioFbankConfig(sampling_rate=24000, n_mels=100, n_fft=1024, hop_length=256))

audio_path = './test.wav'
audio, sr = librosa.load(audio_path, sr=24000)

audio = torch.from_numpy(audio)
logmel = fbank.extract(audio, sr)
logmel_np = logmel.numpy() if hasattr(logmel, "numpy") else logmel


plt.figure(figsize=(10, 4))
plt.imshow(logmel_np.T, 
           aspect="auto", 
           origin="lower", 
           interpolation="none")
plt.colorbar(format="%+2.0f dB")
plt.title("Log-Mel Spectrogram")
plt.xlabel("Frames (Time)")
plt.ylabel("Mel bins")
plt.tight_layout()
plt.show()

In [ ]:
from tokenizerown import LibriTTSTokenizer

text = "Hey... 2015 <filler>"

tokenizer = LibriTTSTokenizer(
    special_tokens=["<filler>"],
    token_file="./vocab_small.txt",
    lowercase=True,
    oov_policy="skip",        # OOV은 버림 (또는 "use_unk", "error")
    unk_token="[UNK]",        # oov_policy="use_unk"일 때만 필요
)
tokens = tokenizer.texts_to_tokens([text])
token_ids = tokenizer.texts_to_token_ids([text])
print(len(token_ids[0]), tokens, token_ids)